## Producer A

In [ ]:
import websocket
import json
from kafka import KafkaProducer
import argparse
import logging
import time
from datetime import datetime

def on_message(ws, message):

    data = json.loads(message)
    logging.info(f"Received message: {data}")
    if 'data' in data:
        quote= {
            "price": data['data'][-1]['p'],
            "s": data['data'][-1]['s'],
            'event_time': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            "v": data['data'][-1]['v']
        }
        producer.send(KAFKA_TOPIC, value=quote)
        print(quote)
        logging.info(f"Sent: {quote}")
            
    time.sleep(5)

def on_error(ws, error):
    print(error)

def on_close(ws):
    print("### closed ###")

def on_open(ws):
    # ws.send('{"type":"subscribe","symbol":"AAPL"}')
    # ws.send('{"type":"subscribe","symbol":"AMZN"}')
    ws.send('{"type":"subscribe","symbol":"BINANCE:BTCUSDT"}')
    # ws.send('{"type":"subscribe","symbol":"IC MARKETS:1"}')

if __name__ == "__main__":

    API_KEY = "d05r249r01qgqsu91h9gd05r249r01qgqsu91ha0"

    # Define Kafka server and topic
    KAFKA_SERVER = '8e35f34f36db:9093'
    KAFKA_TOPIC = 'kafka-spark-example'


    # Initialize the Kafka producer
    producer = KafkaProducer(
        bootstrap_servers=KAFKA_SERVER,
        value_serializer=lambda v: json.dumps(v).encode('utf-8'),  # serialize data as JSON
    )
    
    websocket.enableTrace(True)
    ws = websocket.WebSocketApp("wss://ws.finnhub.io?token=d05r249r01qgqsu91h9gd05r249r01qgqsu91ha0",
                              on_message = on_message,
                              on_error = on_error,
                              on_close = on_close)
    ws.on_open = on_open
    ws.run_forever()